In [ ]:
! pip install transformers

In [ ]:
! pip install datasets

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [2]:
import contextlib
from google.colab import files
import io

with contextlib.redirect_stdout(io.StringIO()):
    files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c feedback-prize-effectiveness

! unzip /content/feedback-prize-effectiveness.zip -d data

In [5]:
import pandas as pd
from transformers import AutoTokenizer

In [13]:
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_train = pd.read_csv('/content/data/train.csv')
df_test = pd.read_csv('/content/data/test.csv')

In [ ]:
df_train['text'] = df_train['discourse_type'] + " " + df_train['discourse_text']
df_test['text'] = df_test['discourse_type'] + " " + df_test['discourse_text']

Label Encoding

In [ ]:
le = LabelEncoder()

le.fit(df_train['discourse_effectiveness'])
df_train['label'] = le.transform(df_train['discourse_effectiveness'])

Train Validation Split

In [34]:
from sklearn.model_selection import train_test_split
val_size = 0.2

df_train_sub = df_train[['text', 'label']]

train_df, val_df = train_test_split(df_train_sub, test_size=val_size, random_state=42)

Convert Pandas Dataframe to Arrow

In [35]:
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

Tokenize the Datasets

In [37]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_val = val_ds.map(preprocess_function, batched=True)

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()